In [1]:
from unityagents import UnityEnvironment
import numpy as np
import matplotlib.pyplot as plt
import torch
from networkforall import Network

env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis.exe")

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [2]:
actor_0 = Network(24, 512, 256, 2, actor=True)
actor_1 = Network(24, 512, 256, 2, actor=True)

checkpoint = torch.load('model_final/episode-8500.pt')
actor_0.load_state_dict(checkpoint[0]['actor_params'])
actor_1.load_state_dict(checkpoint[1]['actor_params'])
actor_0.eval()
actor_1.eval()

Network(
  (fc1): Linear(in_features=24, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=2, bias=True)
)

Note that when invoking the trained agent, only **actors** are needed. 

In [4]:
rewards_overall = np.zeros(2)

for episode in range(100):
    env_info = env.reset(train_mode=True)[brain_name]
    states = env_info.vector_observations
    obs = [agent_obs for agent_obs in states]
    
    for episode_t in range(1000):

        agent_0_action = actor_0(torch.from_numpy(obs[0]).float()).detach().numpy()
        agent_1_action = actor_1(torch.from_numpy(obs[1]).float()).detach().numpy()
        
        actions_array = np.stack([agent_0_action,agent_1_action])
        
        env_info = env.step(actions_array)[brain_name]  

        next_states = env_info.vector_observations
        next_obs = [agent_obs for agent_obs in next_states]

        rewards = np.array(env_info.rewards)
        
        dones = np.array(env_info.local_done)

        obs = next_obs
        
        rewards_overall += rewards
        
        if np.any(dones):
            break
            
print(rewards_overall/(episode+1)) # average rewards

[1.87870003 1.88530003]


In [5]:
env.close()